# Case Study Data-Driven Insights: Funnel and Cohort Analysis for Business Intelligence
## Hijir Della Wirasti

In [1]:
# Import Libraries
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')


# Load File

# Funnel

In [2]:
df_funnel = pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vQOFqPakBTJoVnHGhvKI7i0x84LfhjhwYKS2lt_5TjY9fO5CO3NNx4MQy4A2kLdz7JURWeWQxODKzCd/pub?output=xlsx')
df_funnel.head()

,event_id,user_id,age,gender,city,country,session_id,sequence_number,created_at,browser,traffic_source,event_type
0,555311,42922.0,46.0,M,Bogatynia,Poland,f65f7f3f-1078-45da-a7af-2dd41a894e5d,3,2023-01-19 22:54:38 UTC,Other,Email,cart
1,714816,55343.0,16.0,M,Bogatynia,Poland,cc3509fc-641b-4632-aca7-4949209b9afb,4,2023-11-26 17:12:19 UTC,Safari,Email,cart
2,464405,35775.0,46.0,F,Bogatynia,Poland,16f8b065-9b2e-47bf-b8e7-793060cfbe04,9,2023-09-27 02:46:04 UTC,Firefox,Adwords,cart
3,649908,50243.0,47.0,M,Bogatynia,Poland,8b145bdf-4ddd-473a-8095-c35504afa112,3,2023-04-16 08:50:42 UTC,Chrome,Adwords,cart
4,874098,67437.0,30.0,M,Zgorzelec,Poland,4871ce1f-0b5c-446d-aa7c-016c959cbe56,3,2023-05-18 04:24:49 UTC,Firefox,Email,cart


In [3]:
df_funnel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418279 entries, 0 to 418278
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   event_id         418279 non-null  int64  
 1   user_id          265687 non-null  float64
 2   age              265687 non-null  float64
 3   gender           265687 non-null  object 
 4   city             262839 non-null  object 
 5   country          265687 non-null  object 
 6   session_id       418279 non-null  object 
 7   sequence_number  418279 non-null  int64  
 8   created_at       418279 non-null  object 
 9   browser          418279 non-null  object 
 10  traffic_source   418279 non-null  object 
 11  event_type       418279 non-null  object 
dtypes: float64(2), int64(2), object(8)
memory usage: 38.3+ MB


In [4]:
df_funnel.describe()

,event_id,user_id,age,sequence_number
count,4.182790e+05,265687.000000,265687.000000,418279.000000
mean,1.088930e+06,49931.155348,41.025319,3.940023
std,6.835370e+05,29001.961975,17.092576,2.855708
min,1.600000e+01,2.000000,12.000000,1.000000
25%,5.066330e+05,24718.000000,26.000000,2.000000
50%,1.019051e+06,50062.000000,41.000000,3.000000
75%,1.651511e+06,75007.000000,56.000000,5.000000
max,2.420896e+06,99999.000000,70.000000,13.000000


## Data Preprocessing

In [5]:
# see the number of missing values from the data frame
df_funnel_missing = df_funnel.isnull().sum().reset_index()
df_funnel_missing.columns = ['feature','missing_value']
df_funnel_missing

,feature,missing_value
0,event_id,0
1,user_id,152592
2,age,152592
3,gender,152592
4,city,155440
5,country,152592
6,session_id,0
7,sequence_number,0
8,created_at,0
9,browser,0


In [6]:
# Checking for missing values and their percentages
df_funnel_missing = df_funnel.isnull().sum().reset_index()
df_funnel_missing.columns = ['feature', 'missing_value']
df_funnel_missing['missing_percentage'] = (df_funnel_missing['missing_value'] / len(df_funnel)) * 100

# Format the percentage column to include the '%' symbol
df_funnel_missing['missing_percentage'] = df_funnel_missing['missing_percentage'].apply(lambda x: f"{x:.2f}%")

# Display the DataFrame with missing values and percentages
print(df_funnel_missing)



            feature  missing_value missing_percentage
0          event_id              0              0.00%
1           user_id         152592             36.48%
2               age         152592             36.48%
3            gender         152592             36.48%
4              city         155440             37.16%
5           country         152592             36.48%
6        session_id              0              0.00%
7   sequence_number              0              0.00%
8        created_at              0              0.00%
9           browser              0              0.00%
10   traffic_source              0              0.00%
11       event_type              0              0.00%


Untuk funnel analysis, penanganan missing value tergantung pada kolom yang hilang dan perannya dalam analisis. Funnel analysis biasanya digunakan untuk melacak perjalanan pengguna melalui tahapan tertentu dalam sebuah proses (misalnya, **visitors → signups → purchases**). Berikut adalah cara menangani missing value:

---

### **Kolom Penting untuk Funnel Analysis**
Kolom penting yang sering digunakan:
1. **user_id**: Mengidentifikasi pengguna unik. Wajib ada.
2. **event_type**: Menunjukkan tahapan dalam funnel (e.g., "visited", "signed_up", "purchased"). Wajib ada.
3. **created_at**: Waktu aktivitas untuk menentukan urutan atau waktu retensi. Wajib ada.




In [7]:
# Mengisi nilai kosong pada kolom user_id dengan 0
df_funnel['user_id'].fillna(0, inplace=True)

In [8]:
# Melihat nilai unik dari kolom event_type
unique_event_types = df_funnel['event_type'].unique()


In [9]:
# Melihat isi dari setiap event_type dan jumlahnya
event_type_counts = df_funnel['event_type'].value_counts()

# Menampilkan hasil
print(event_type_counts)


event_type
product     182825
cart        139267
purchase     50185
home         24234
cancel       21768
Name: count, dtype: int64


---

### **Kolom Opsional**
Kolom-kolom ini mungkin digunakan untuk analisis granular (segmentasi lebih detail), tetapi tidak wajib:
- **age**, **gender**, **city**, **country** (data demografi).
- **traffic_source**, **browser** (informasi sumber dan perangkat).

**Penanganan**:
1. **Imputasi dengan nilai default**:
   - **age**: Median atau rata-rata.
   - **gender**: Modus (nilai paling umum).
   - **city/country**: Modus.
   - **traffic_source/browser**: Isi dengan "Unknown".



In [10]:
# Fill missing optional columns with default values
df_funnel['age'].fillna(df_funnel['age'].median(), inplace=True)
df_funnel['gender'].fillna(df_funnel['gender'].mode()[0], inplace=True)
df_funnel['city'].fillna(df_funnel['city'].mode()[0], inplace=True)
df_funnel['country'].fillna(df_funnel['country'].mode()[0], inplace=True)
df_funnel['traffic_source'].fillna(df_funnel['traffic_source'].mode()[0], inplace=True)
df_funnel['browser'].fillna(df_funnel['browser'].mode()[0], inplace=True)


In [11]:
# Check missing values after imputations
print(df_funnel.isnull().sum())


event_id           0
user_id            0
age                0
gender             0
city               0
country            0
session_id         0
sequence_number    0
created_at         0
browser            0
traffic_source     0
event_type         0
dtype: int64


In [12]:
df_funnel.duplicated().sum()

0

In [13]:
df_funnel.to_excel('funnel_analysis.xlsx', index=False)

# Unduh file Excel
from google.colab import files
files.download('funnel_analysis.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Cohort

In [ ]:
df_cohort = pd.read_excel('https://docs.google.com/spreadsheets/d/e/2PACX-1vQVz3pAS2YM90tRffL5NFyIARc9EAG1MBIsWW-QQyXnhAYN2qUfHwUlewYxhWDa12hnzHABL6PinJBI/pub?output=xlsx')
df_cohort.head()

,order_id,order_date,user_id,user_name,age,gender,product_id,product_name,category,quantity,sale_price,cost
0,102234,2024-10-22,81494,Sheri Ramos,31,F,8038,Embroidered Capri Set - Sizes: 1X2X3X4X,Clothing Sets,1,22.990000,13.862970
1,72773,2024-03-02,57833,Judy Beltran,56,F,8032,Aeropostale Womens; Juniors Long Sleeve Logo G...,Clothing Sets,1,34.500000,21.010500
2,123719,2024-03-01,98736,Erin Mora,55,F,8030,Calvin Klein Women's MSY Velour Pant,Clothing Sets,1,50.000000,32.900000
3,14065,2024-05-22,11186,Shelly Hill,25,F,8027,Only Necessities Plus Size Peachskin 3 piece P...,Clothing Sets,1,54.990002,35.798491
4,37640,2024-10-07,29909,Tina Davis,26,F,8049,VIOLET BLOUSE GAUCHO PALAZZO SET ASYM - FITS (...,Clothing Sets,1,57.990002,34.098121


In [ ]:
df_cohort.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19163 entries, 0 to 19162
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   order_id      19163 non-null  int64         
 1   order_date    19163 non-null  datetime64[ns]
 2   user_id       19163 non-null  int64         
 3   user_name     19163 non-null  object        
 4   age           19163 non-null  int64         
 5   gender        19163 non-null  object        
 6   product_id    19163 non-null  int64         
 7   product_name  19162 non-null  object        
 8   category      19163 non-null  object        
 9   quantity      19163 non-null  int64         
 10  sale_price    19163 non-null  float64       
 11  cost          19163 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(5), object(4)
memory usage: 1.8+ MB


In [ ]:
df_cohort.describe()

,order_id,order_date,user_id,age,product_id,quantity,sale_price,cost
count,19163.000000,19163,19163.000000,19163.000000,19163.000000,19163.0,19163.000000,19163.000000
mean,63115.468455,2024-07-29 17:22:24.528518656,50234.590617,41.215624,15305.692585,1.0,59.387922,28.565440
min,5.000000,2024-01-01 00:00:00,3.000000,12.000000,1.000000,1.0,0.020000,0.008300
25%,31572.000000,2024-05-03 00:00:00,25133.000000,26.000000,8044.000000,1.0,24.299999,11.380500
50%,63735.000000,2024-08-12 00:00:00,50599.000000,41.000000,16095.000000,1.0,39.990002,19.755061
75%,94477.000000,2024-11-01 00:00:00,75259.000000,56.000000,22625.000000,1.0,69.800003,34.340000
max,125326.000000,2024-12-31 00:00:00,99987.000000,70.000000,29120.000000,1.0,999.000000,557.151002
std,36335.649328,NaN,28984.285353,17.112735,8425.189694,0.0,65.304680,30.260888


In [ ]:
# Checking for missing values and their percentages
df_cohort_missing = df_cohort.isnull().sum().reset_index()
df_cohort_missing.columns = ['feature', 'missing_value']
df_cohort_missing['missing_percentage'] = (df_cohort_missing['missing_value'] / len(df_cohort)) * 100

# Format the percentage column to include the '%' symbol
df_cohort_missing['missing_percentage'] = df_cohort_missing['missing_percentage'].apply(lambda x: f"{x:.2f}%")

# Display the DataFrame with missing values and percentages
print(df_cohort_missing)

         feature  missing_value missing_percentage
0       order_id              0              0.00%
1     order_date              0              0.00%
2        user_id              0              0.00%
3      user_name              0              0.00%
4            age              0              0.00%
5         gender              0              0.00%
6     product_id              0              0.00%
7   product_name              1              0.01%
8       category              0              0.00%
9       quantity              0              0.00%
10    sale_price              0              0.00%
11          cost              0              0.00%


Kolom product_name memiliki 1 nilai hilang (0.01%). Ini kecil dan tidak akan memengaruhi analisis jika dihapus atau diisi dengan nilai default. <br>

Rekomendasi:
Hapus baris dengan nilai hilang pada product_name karena proporsi missing value sangat kecil.

In [ ]:
# Drop rows where product_name is missing
df_coclean = df_cohort.dropna(subset=['product_name'])


In [ ]:
# Check missing values after imputations
print(df_coclean.isnull().sum())

order_id        0
order_date      0
user_id         0
user_name       0
age             0
gender          0
product_id      0
product_name    0
category        0
quantity        0
sale_price      0
cost            0
dtype: int64


In [ ]:
df_coclean.duplicated().sum()

0

In [ ]:
df_coclean.to_csv('cohort.csv', index=False)

# Unduh file Excel
from google.colab import files
files.download('cohort.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
df_coclean.to_excel('cohort.xlsx', index=False)

# Unduh file Excel
from google.colab import files
files.download('cohort.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>